In [2]:
from datasets import load_dataset

# Load the ACLSum dataset
dataset = load_dataset("sobamchan/aclsum", split="train")

# dataset = load_dataset("sobamchan/aclsum", "challenge", split="train")

# Display a sample entry
print(dataset[0])

c:\Users\ASUS\xStyNWx\Documents\BINUS University\Academic Courses\Semester 4\Natural Language Processing\Project\Repo\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'id': 'P06-1112', 'document': 'In this paper , we explore correlation of dependency relation paths to rank candidate answers in answer extraction . Using the correlation measure , we compare dependency relations of a candidate answer and mapped question phrases in sentence with the corresponding relations in question . Different from previous studies , we propose an approximate phrase mapping algorithm and incorporate the mapping score into the correlation measure . The correlations are further incorporated into a Maximum Entropy-based ranking model which estimates path weights from training . Experimental results show that our method significantly outperforms state-ofthe-art syntactic relation-based methods by up to 20 % in MRR . Answer Extraction is one of basic modules in open domain Question Answering ( QA ) . It is to further process relevant sentences extracted with Passage / Sentence Retrieval and pinpoint exact answers using more linguistic-motivated analysis . Since QA turns 

In [3]:
print(dataset[0]['approach'])

They propose a statistical method which takes correlations of dependency relation paths computed by the Dynamic Time Wrapping algorithm into account for ranking candidate answers.


In [4]:
from transformers import AutoTokenizer

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

# Function to preprocess the data
def preprocess_data(example):
    inputs = example['document']
    targets = example['outcome']
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(targets, max_length=150, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing
tokenized_dataset = dataset.map(preprocess_data, batched=True)


In [5]:
from transformers import T5ForConditionalGeneration

# Load the pre-trained T5 model
model = T5ForConditionalGeneration.from_pretrained("t5-small")


In [6]:
from transformers import Seq2SeqTrainingArguments

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True
)


c:\Users\ASUS\xStyNWx\Documents\BINUS University\Academic Courses\Semester 4\Natural Language Processing\Project\Repo\venv\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [7]:
from transformers import DataCollatorForSeq2Seq

# Initialize data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [8]:
from transformers import Seq2SeqTrainer

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,  # For demonstration; ideally, use a separate validation set
    tokenizer=tokenizer,
    data_collator=data_collator
)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_19432\3019943128.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
# Train the model
trainer.train()


In [ ]:
# Evaluate the model
results = trainer.evaluate()

# Print evaluation results
print(results)


In [9]:
# Function to generate summary
def generate_summary(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs.input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Example usage
sample_paper = dataset[0]['document']
print("Original Paper:\n", sample_paper)
print("\nGenerated Summary:\n", generate_summary(sample_paper))


Original Paper:
 In this paper , we explore correlation of dependency relation paths to rank candidate answers in answer extraction . Using the correlation measure , we compare dependency relations of a candidate answer and mapped question phrases in sentence with the corresponding relations in question . Different from previous studies , we propose an approximate phrase mapping algorithm and incorporate the mapping score into the correlation measure . The correlations are further incorporated into a Maximum Entropy-based ranking model which estimates path weights from training . Experimental results show that our method significantly outperforms state-ofthe-art syntactic relation-based methods by up to 20 % in MRR . Answer Extraction is one of basic modules in open domain Question Answering ( QA ) . It is to further process relevant sentences extracted with Passage / Sentence Retrieval and pinpoint exact answers using more linguistic-motivated analysis . Since QA turns to find exact a

In [10]:
# Function to generate summary
def generate_summary(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs.input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Example usage
sample_paper = dataset[10]['document']
print("Original Paper:\n", sample_paper)
print("\nGenerated Summary:\n", generate_summary(sample_paper))


Original Paper:
 Automatically extracting social meaning and intention from spoken dialogue is an important task for dialogue systems and social computing . We describe a system for detecting elements of interactional style : whether a speaker is awkward , friendly , or flirtatious . We create and use a new spoken corpus of 991 4-minute speed-dates . Participants rated their interlocutors for these elements of style . Using rich dialogue , lexical , and prosodic features , we are able to detect flirtatious , awkward , and friendly styles in noisy natural conversational data with up to 75 % accuracy , compared to a 50 % baseline . We describe simple ways to extract relatively rich dialogue features , and analyze which features performed similarly for men and women and which were gender-specific . How can we extract social meaning from speech , deciding if a speaker is particularly engaged in the conversation , is uncomfortable or awkward , or is particularly friendly and flirtatious ? U

Original Paper:
 Automatically extracting social meaning and intention from spoken dialogue is an important task for dialogue systems and social computing . We describe a system for detecting elements of interactional style : whether a speaker is awkward , friendly , or flirtatious . We create and use a new spoken corpus of 991 4-minute speed-dates . Participants rated their interlocutors for these elements of style . Using rich dialogue , lexical , and prosodic features , we are able to detect flirtatious , awkward , and friendly styles in noisy natural conversational data with up to 75 % accuracy , compared to a 50 % baseline . We describe simple ways to extract relatively rich dialogue features , and analyze which features performed similarly for men and women and which were gender-specific . How can we extract social meaning from speech , deciding if a speaker is particularly engaged in the conversation , is uncomfortable or awkward , or is particularly friendly and flirtatious ? Understanding these meanings and how they are signaled in language is an important sociolinguistic task in itself . Extracting them automatically from dialogue speech and text is crucial for developing socially aware computing systems for tasks such as detection of interactional problems or matching conversational style , and will play an important role in creating more natural dialogue agents ( Pentland , 2005 ; Nass and Brave , 2005 ; Brave et al . , 2005 ) . Cues for social meaning permeate speech at every level of linguistic structure . Acoustic cues such as low and high F0 or energy and spectral tilt are important in detecting emotions such as annoyance , anger , sadness , or boredom ( Ang et al . , 2002 ; Lee and Narayanan , 2002 ; Liscombe et al . , 2003 ) , speaker characteristics such as charisma ( Rosenberg and Hirschberg , 2005 ) , or personality features like extroversion ( Mairesse et al . , 2007 ; Mairesse and Walker , 2008 ) . Lexical cues to social meaning abound . Speakers with links to depression or speakers who are under stress use more first person singular pronouns ( Rude et al . , 2004 ; Pennebaker and Lay , 2002 ; Cohn et al . , 2004 ) , positive emotion words are cues to agreeableness ( Mairesse et al . , 2007 ) , and negative emotion words are useful cues to deceptive speech ( Newman et al . , 2003 ) . The number of words in a sentence can be a useful feature for extroverted personality ( Mairesse et al . , 2007 ) . Finally , dialog features such as the presence of disfluencies can inform listeners about speakers ' problems in utterance planning or about confidence ( Brennan and Williams , 1995 ; Brennan and Schober , 2001 ) . Our goal is to see whether cues of this sort are useful in detecting particular elements of conversational style and social intention ; whether a speaker in a speed-dating conversation is judged by the interlocutor as friendly , awkward , or flirtatious . The results presented here should be regarded with some caution . The sample is not a random sample of English speakers or American adults , and speed dating is not a natural context for expressing every conversational style . Therefore , a wider array of studies across populations and genres would be required before a more general theory of conversational styles is established . On the other hand , the presented results may under-reflect the relations being captured . The quality of recordings and coarse granularity ( 1 second ) of the time-stamps likely cloud the relations , and as the data is cleaned and improved , we expect the associations to only grow stronger . Caveats aside , we believe the evidence indicates that the perception of several types of conversational style have relatively clear signals across genders , but with some additional gender contextualization . Both genders convey flirtation by laughing more , speaking faster , and using higher and more variable pitch . Both genders convey friendliness by laughing more , and using collaborative completions . However , we do find gender differences ; men asl more questions when ( labeled as ) flirting , women ask fewer . Men labeled as flirting are softer , but women labeled as flirting are louder . Women flirt-ing swear more , while men are more likely to use sexual vocabulary . Gender differences exist as well for the other variables . Men labeled as friendly use you while women labeled as friendly use I. Friendly women are very disfluent ; friendly men are not . While the features for friendly and flirtatious speech overlap , there are clear differences . Men speaker faster and with higher f0 ( min ) in flirtatious speech , but not faster and with lower f0 ( min ) in friendly speech . For men , flirtatious speech involves more questions and repair questions , while friendly speech does not . For women , friendly speech is more disfluent than flirtatious speech , and has more collaborative style ( completions , repair questions , appreciations ) . We also seem to see a model of collaborative conversational style ( probably related to the collaborative floor of Edelsky ( 1981 ) and Coates ( 1996 ) ) , cued by the use of more collaborative completions , repair questions and other questions , you , and laughter . These collaborative techniques were used by both women and men who were labeled as friendly , and occurred less with men labeled as awkward . Women themselves displayed more of this collaborative conversational style when they labeled the men as friendly . For women only , collaborative style included appreciations ; while for men only , collaborative style included overlaps . In addition to these implications for social science , our work has implications for the extraction of meaning in general . A key focus of our work was on ways to extract useful dialog act and disfluency features ( repair questions , backchannels , appreciations , restarts , dispreferreds ) with very shallow methods . These features were indeed extractable and proved to be useful features in classification . We are currently extending these results to predict date outcomes including ' liking ' , extending work such as Madan and Pentland ( 2006 ) .

Generated Summary:
 Using rich dialogue, lexical, and prosodic features, we are able to detect flirtatious, awkward, and friendly styles in noisy natural conversational data with up to 75 % accuracy.

In [ ]:
# Save the fine-tuned model
model.save_pretrained("saved_model")
tokenizer.save_pretrained("saved_model")


In [ ]:
from transformers import T5ForConditionalGeneration, AutoTokenizer

# Load saved model
model = T5ForConditionalGeneration.from_pretrained("saved_model")
tokenizer = AutoTokenizer.from_pretrained("saved_model")


c:\Users\ASUS\xStyNWx\Documents\BINUS University\Academic Courses\Semester 4\Natural Language Processing\Project\Repo\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
# Function to generate summary
def generate_summary(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs.input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Example usage
sample_paper = dataset[0]['document']
print("Original Paper:\n", sample_paper)
print("\nGenerated Summary:\n", generate_summary(sample_paper))


Original Paper:
 In this paper , we explore correlation of dependency relation paths to rank candidate answers in answer extraction . Using the correlation measure , we compare dependency relations of a candidate answer and mapped question phrases in sentence with the corresponding relations in question . Different from previous studies , we propose an approximate phrase mapping algorithm and incorporate the mapping score into the correlation measure . The correlations are further incorporated into a Maximum Entropy-based ranking model which estimates path weights from training . Experimental results show that our method significantly outperforms state-ofthe-art syntactic relation-based methods by up to 20 % in MRR . Answer Extraction is one of basic modules in open domain Question Answering ( QA ) . It is to further process relevant sentences extracted with Passage / Sentence Retrieval and pinpoint exact answers using more linguistic-motivated analysis . Since QA turns to find exact a

In [12]:
# Function to generate summary
def generate_summary(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs.input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Example usage
sample_paper = "Recent developments in sequence-to-sequence learning with neural networks have considerably improved the quality of automatically generated text summaries and document keywords, stipulating the need for even bigger training corpora. Metadata of research articles are usually easy to find online and can be used to perform research on various tasks. In this paper, we introduce two huge datasets for text summarization (OAGSX) and keyword generation (OAGKX) research, containing 34 million and 23 million records, respectively. The data were retrieved from the Open Academic Graph which is a network of research profiles and publications. We carefully processed each record and also tried several extractive and abstractive methods of both tasks to create performance baselines for other researchers. We further illustrate the performance of those methods previewing their outputs. In the near future, we would like to apply topic modeling on the two sets to derive subsets of research articles from more specific disciplines."
print("Original Paper:\n", sample_paper)
print("\nGenerated Summary:\n", generate_summary(sample_paper))


Original Paper:
 Recent developments in sequence-to-sequence learning with neural networks have considerably improved the quality of automatically generated text summaries and document keywords, stipulating the need for even bigger training corpora. Metadata of research articles are usually easy to find online and can be used to perform research on various tasks. In this paper, we introduce two huge datasets for text summarization (OAGSX) and keyword generation (OAGKX) research, containing 34 million and 23 million records, respectively. The data were retrieved from the Open Academic Graph which is a network of research profiles and publications. We carefully processed each record and also tried several extractive and abstractive methods of both tasks to create performance baselines for other researchers. We further illustrate the performance of those methods previewing their outputs. In the near future, we would like to apply topic modeling on the two sets to derive subsets of researc